In [1]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras

In [110]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [88]:
from sklearn.model_selection import train_test_split

In [2]:
data_path = "D:\\SourceLocalization.csv"

In [3]:
data = pd.read_csv(data_path)

In [5]:
data.shape

(1000000, 9)

In [6]:
set1_path = "D:/DataIITM/data1.csv"

In [7]:
set1 = pd.read_csv(set1_path)

In [8]:
set1.head()

,Unnamed: 0,Transmitter Location,Receiver Location,Magnetic_Vector,Transmitter Orientation,Reciever Orientation,target_x,target_y,target_z
0,0,[10. 0. 18.],[-10. -10. -10.],"(1.52637256e-06, 7.26308328e-07, -4.59565699e-07)","(-90.0, -30.0, -180.0)","(-90.0, -30.0, -180.0)",-28.275363,9.495643,-19.857911
1,1,[10. 0. 18.],[-10. -10. -10.],"(-3.30903163e-07, -3.45990308e-07, -1.68503157...","(-90.0, -30.0, -180.0)","(-90.0, -30.0, -90.0)",-30.426208,5.947252,17.968752
2,2,[10. 0. 18.],[-10. -10. -10.],"(-1.39218774e-06, -9.58723245e-07, 4.59565699e...","(-90.0, -30.0, -180.0)","(-90.0, -30.0, 0.0)",-24.579856,-26.007884,-1.849500
3,3,[10. 0. 18.],[-10. -10. -10.],"(4.65087978e-07, 1.13575391e-07, 1.68503157e-06)","(-90.0, -30.0, -180.0)","(-90.0, -30.0, 90.0)",-27.668203,6.177014,-21.916091
4,4,[10. 0. 18.],[-10. -10. -10.],"(1.52637256e-06, 7.26308328e-07, -4.59565699e-07)","(-90.0, -30.0, -180.0)","(-90.0, -30.0, 180.0)",-30.746942,9.289743,15.884780


In [9]:
set1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 9 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0               int64  
 1   Transmitter Location     object 
 2   Receiver Location        object 
 3   Magnetic_Vector          object 
 4   Transmitter Orientation  object 
 5   Reciever Orientation     object 
 6   target_x                 float64
 7   target_y                 float64
 8   target_z                 float64
dtypes: float64(3), int64(1), object(5)
memory usage: 343.3+ MB


In [10]:
set1.drop(columns = ['Unnamed: 0'] , inplace = True)

In [12]:
set1.head()

,Transmitter Location,Receiver Location,Magnetic_Vector,Transmitter Orientation,Reciever Orientation,target_x,target_y,target_z
0,[10. 0. 18.],[-10. -10. -10.],"(1.52637256e-06, 7.26308328e-07, -4.59565699e-07)","(-90.0, -30.0, -180.0)","(-90.0, -30.0, -180.0)",-28.275363,9.495643,-19.857911
1,[10. 0. 18.],[-10. -10. -10.],"(-3.30903163e-07, -3.45990308e-07, -1.68503157...","(-90.0, -30.0, -180.0)","(-90.0, -30.0, -90.0)",-30.426208,5.947252,17.968752
2,[10. 0. 18.],[-10. -10. -10.],"(-1.39218774e-06, -9.58723245e-07, 4.59565699e...","(-90.0, -30.0, -180.0)","(-90.0, -30.0, 0.0)",-24.579856,-26.007884,-1.849500
3,[10. 0. 18.],[-10. -10. -10.],"(4.65087978e-07, 1.13575391e-07, 1.68503157e-06)","(-90.0, -30.0, -180.0)","(-90.0, -30.0, 90.0)",-27.668203,6.177014,-21.916091
4,[10. 0. 18.],[-10. -10. -10.],"(1.52637256e-06, 7.26308328e-07, -4.59565699e-07)","(-90.0, -30.0, -180.0)","(-90.0, -30.0, 180.0)",-30.746942,9.289743,15.884780


In [14]:
target_columns = []
training_columns = []
for i in set1.columns:
    if i[0].lower() == 't' :
        target_columns.append(i)
    else : 
        training_columns.append(i)

In [15]:
training_columns

['Receiver Location', 'Magnetic_Vector', 'Reciever Orientation']

In [16]:
target_columns

['Transmitter Location',
 'Transmitter Orientation',
 'target_x',
 'target_y',
 'target_z']

In [17]:
training_set = set1[training_columns]

In [19]:
target_set = set1[target_columns]

In [20]:
training_set.head()

,Receiver Location,Magnetic_Vector,Reciever Orientation
0,[-10. -10. -10.],"(1.52637256e-06, 7.26308328e-07, -4.59565699e-07)","(-90.0, -30.0, -180.0)"
1,[-10. -10. -10.],"(-3.30903163e-07, -3.45990308e-07, -1.68503157...","(-90.0, -30.0, -90.0)"
2,[-10. -10. -10.],"(-1.39218774e-06, -9.58723245e-07, 4.59565699e...","(-90.0, -30.0, 0.0)"
3,[-10. -10. -10.],"(4.65087978e-07, 1.13575391e-07, 1.68503157e-06)","(-90.0, -30.0, 90.0)"
4,[-10. -10. -10.],"(1.52637256e-06, 7.26308328e-07, -4.59565699e-07)","(-90.0, -30.0, 180.0)"


## Preparing the Data For Training 

In [66]:
def columns_seperation(feature):
    
    x_com = []
    y_com = []
    z_com = []
    for i in feature:
        ## For reciever location in split function dont use anything as it has seperated by spaces
        x , y , z = list(map(float , i[1:-1].split(','))) 
        x_com.append(x)
        y_com.append(y)
        z_com.append(z)
    return x_com , y_com , z_com

In [50]:
rec_x , rec_y , rec_z = columns_seperation(training_set['Receiver Location'])

In [67]:
mag_x , mag_y , mag_z = columns_seperation(training_set['Magnetic_Vector'])

In [77]:
ori_x , ori_y , ori_z = columns_seperation(training_set['Reciever Orientation'])

In [81]:
data = {
    'rec_x':rec_x , 'rec_y':rec_y , 'rec_z':rec_z,
    'mag_x':mag_x , 'mag_z':mag_y , 'mag_z':mag_z ,
    'roll_r':ori_x , 'pitch_r':ori_y , 'yaw_r':ori_z
}

In [83]:
training_data = pd.DataFrame(data)

In [87]:
target_data = target_set[['target_x' , 'target_y' , 'target_z']]

In [97]:
train_x , test_x , train_y , test_y = train_test_split(training_data , target_data , train_size=0.8 , random_state=42)

In [98]:
train_x , val_x , train_y , val_y = train_test_split(train_x , train_y , train_size=0.8 , random_state=42)

In [99]:
len(train_x) , len(val_x) , len(test_x)

(3200000, 800000, 1000000)

In [102]:
train_x.shape

(3200000, 8)

In [108]:
from keras.models import Sequential
from keras.layers import Dense , Dropout , BatchNormalization , Input
from keras.losses import mean_squared_error 
from keras.optimizers import RMSprop
from keras.activations import selu
from keras.metrics import R2Score

In [154]:
model = Sequential()
model.add(Input(shape=(8,)))
model.add(Dense(8 , activation= 'linear'))
# model.add(BatchNormalization())
model.add(Dense(16 , activation= 'linear'))
# model.add(BatchNormalization())
model.add(Dense(24 , activation= 'linear'))
# model.add(BatchNormalization())
# model.add(Dropout(0.1))
model.add(Dense(56 , activation= 'selu'))
# model.add(Dense(64 , activation= 'linear'))
# model.add(Dense(64 , activation= 'linear'))
# model.add(Dense(56 , activation= 'linear'))
model.add(Dense(24 , activation= 'linear'))
# model.add(BatchNormalization())
model.add(Dense(16 , activation= 'linear'))
model.add(Dense(8 , activation= 'linear'))
model.add(Dense(3 , activation='linear'))

In [155]:
model.compile(loss = mean_squared_error , metrics=[R2Score()])

In [156]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_94 (Dense)            (None, 8)                 72        
                                                                 
 dense_95 (Dense)            (None, 16)                144       
                                                                 
 dense_96 (Dense)            (None, 24)                408       
                                                                 
 dense_97 (Dense)            (None, 56)                1400      
                                                                 
 dense_98 (Dense)            (None, 24)                1368      
                                                                 
 dense_99 (Dense)            (None, 16)                400       
                                                                 
 dense_100 (Dense)           (None, 8)               

In [157]:
history = model.fit(x = train_x.values , y = train_y.values , validation_data= [val_x.values , val_y.values] , epochs=2)

Epoch 1/2
100000/100000 [==============================] - 919s 9ms/step - loss: 120.6849 - r2_score: 0.7188 - val_loss: 82.0765 - val_r2_score: 0.8085
Epoch 2/2
100000/100000 [==============================] - 582s 6ms/step - loss: 77.7025 - r2_score: 0.8190 - val_loss: 72.9446 - val_r2_score: 0.8297


In [180]:
import json

In [185]:
model.save('D:/DataIITM/Model')

INFO:tensorflow:Assets written to: D:/DataIITM/Model\assets


INFO:tensorflow:Assets written to: D:/DataIITM/Model\assets
